In [1]:
%display latex
from pylie import *
from collections.abc import Iterable
import functools
from operator import mul, sub
from pprint import pprint

In [2]:
def reduce(e1: Differential_Polynomial,e2: Differential_Polynomial, context:Context)->Differential_Polynomial:
    # XXX except only function names in context list
    assert e2.is_monic()
    for t, d, c, in zip (e1.terms(), e1.derivatives(), e1.coefficients()):
        ld = e2.Lder()
        def _order (der):
            if der != 1:
                return pylie.order_of_derivative(der)
            else :
                return [0]*len(context._independent)
        e2_order = _order (ld)
        e1_order = _order (d)
        dif = []
        for a, b in zip (e1_order, e2_order):
            dif += [a - b]
        try:
            if ld.operator().function() != d.operator().function():
                continue
        except AttributeError:
            continue            
        if all (map (lambda h: h == 0, dif)) :
            # XXX: do we need that in homogenous environmeent?
            inter = sum(_ for _ in e1.terms()) - c * sum (_ for _ in e2.terms())
            dp = Differential_Polynomial(inter.simplify_full().expand(), context)
            assert dp.is_monic()
            return dp
        elif  all (map (lambda h: h >= 0, dif)):
            print ("PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP")
            # get variables which shuld be differentiate
            variables_to_diff = []
            for i in range (len(context._independent)):
                if dif [i] != 0:
                    variables_to_diff.extend ([context._independent[i]]*abs(dif[i]))
            print ("Vorher", variables_to_diff)
            print ("reduce {} with {}".format(e1.Lder(), e2.Lder()))
            print ("e1:", e1.Lterm())
            print ("e2:", e2.Lterm())
            e1._orig.show()
            e2._orig.show()
            print ("after derivation................")
            diff(e2._orig, *variables_to_diff).show()
            e3 = e1._orig - c * diff(e2._orig, *variables_to_diff)
            e3.show()
            e3 = e3.expand().simplify_full()
            e3.show()
            dp =  Differential_Polynomial (e3.expand().simplify_full(), context)
            print ("e3:", dp.Lterm())            
            #for _ in dp._p:
            #    print (":3", _._d, _._coeff)            
            return dp
        else:
            print ("no reduction?")
            pass
    return e1

In [3]:
var("x y")
z  = function ("z")(x,y)
ctx = Context((z,), (x,y))
print("*"*70)
e1 = diff(z,y)-x^2/y^2*diff(z,x)-(x-y)*z(x,y)/y^2
e1_dp = Differential_Polynomial (e1, ctx)
print (e1_dp)
print("*"*70)
f1 = diff(z, x) + z(x,y)/x
f1_dp = Differential_Polynomial (f1, ctx)
f1_dp.is_monic()
print (f1_dp)
print("*"*70)
f2 = diff(z, y) + z(x,y)/y
f2_dp = Differential_Polynomial (f2, ctx)
print (f2_dp)

**********************************************************************
This is the divisor: -x^2/y^2
     Coeff: -x^2/y^2
         d: diff(z(x, y), x)
       out: 1
       res: diff(z(x, y), x)
     Coeff: 1
         d: diff(z(x, y), y)
       out: -y^2/x^2
       res: -y^2*diff(z(x, y), y)/x^2
     Coeff: -(x - y)*z(x, y)/y^2
         d: 1
       out: (x - y)*z(x, y)/x^2
       res: (x - y)*z(x, y)/x^2
-x^2*diff(z(x, y), x)/y^2 - (x - y)*z(x, y)/y^2 + diff(z(x, y), y)
**********************************************************************
This is the divisor: 1
     Coeff: 1
         d: diff(z(x, y), x)
       out: 1.0
       res: 1.0*diff(z(x, y), x)
     Coeff: z(x, y)/x
         d: 1
       out: z(x, y)/x
       res: z(x, y)/x
z(x, y)/x + diff(z(x, y), x)
**********************************************************************
This is the divisor: 1
     Coeff: 1
         d: diff(z(x, y), y)
       out: 1.0
       res: 1.0*diff(z(x, y), y)
     Coeff: z(x, y)/y
         d: 1
       ou

In [4]:
r1 = reduce (e1_dp, f1_dp, ctx)

This is the divisor: -y^2/x^2
     Coeff: -y^2/x^2
         d: diff(z(x, y), y)
       out: 1
       res: diff(z(x, y), y)
     Coeff: -y*z(x, y)/x^2
         d: 1
       out: z(x, y)/y
       res: z(x, y)/y


In [5]:
r1

In [6]:
r1._orig

-y^2*diff(z(x, y), y)/x^2 - y*z(x, y)/x^2

In [7]:
r1._p

[<pylie.JanetBasis.DTerm object at 0x7feea9cad490>,
 <pylie.JanetBasis.DTerm object at 0x7feeab637dc0>]

In [8]:
f2_dp._p

[<pylie.JanetBasis.DTerm object at 0x7feea9c9d940>,
 <pylie.JanetBasis.DTerm object at 0x7feea9c9d9d0>]

In [9]:
r2 = reduce (r1, f2_dp, ctx)

In [10]:
r2

In [11]:
def Lfunc(e):
    return e

In [12]:
def Lcml(e):
    '''Least common left multiply, p.23'''
    return 0

In [13]:
def Lcm(e):
    return 0

In [14]:
def CompleteSystem(S, context):
    return S

In [15]:
def FindIntegrableConditions(S, context):
    return S

In [16]:
def ReduceIntegrabilityConditions(S, context):
    return S

In [17]:
def Reorder (S, context, ascending = False):
    res=sorted(S)
    if ascending :
        res.reverse()
    return res

In [18]:
var ("x y")
z = function("z")(x,y)
w = function("w")(x,y)
ctx = Context((w,z), (x,y), MatrixOrder.Mgrlex)
f1 = Differential_Polynomial(diff(w, y) + x*diff(z,y)/(2*y*(x**2+y)) - w(x,y)/y, ctx)
f1._orig

This is the divisor: 1
     Coeff: 1
         d: diff(w(x, y), y)
       out: 1.0
       res: 1.0*diff(w(x, y), y)
     Coeff: 1/2*x/((x^2 + y)*y)
         d: diff(z(x, y), y)
       out: 1/2*x/((x^2 + y)*y)
       res: 1/2*x*diff(z(x, y), y)/((x^2 + y)*y)
     Coeff: -w(x, y)/y
         d: 1
       out: -w(x, y)/y
       res: -w(x, y)/y


-w(x, y)/y + 1/2*x*diff(z(x, y), y)/((x^2 + y)*y) + diff(w(x, y), y)

In [19]:
f2=Differential_Polynomial(diff(z,x,y) + y*diff(w,y)/x + 2*y*diff(z, x)/x,ctx)
f2._orig

This is the divisor: 1
     Coeff: 1
         d: diff(z(x, y), x, y)
       out: 1.0
       res: 1.0*diff(z(x, y), x, y)
     Coeff: y/x
         d: diff(w(x, y), y)
       out: y/x
       res: y*diff(w(x, y), y)/x
     Coeff: 2*y/x
         d: diff(z(x, y), x)
       out: 2*y/x
       res: 2*y*diff(z(x, y), x)/x


y*diff(w(x, y), y)/x + 2*y*diff(z(x, y), x)/x + diff(z(x, y), x, y)

In [20]:
f3 = Differential_Polynomial(diff(w, x,y) - 2*x*diff(z, x,2)/y - x*diff(w,x)/y**2 ,ctx)
f3._orig

This is the divisor: 1
     Coeff: 1
         d: diff(w(x, y), x, y)
       out: 1.0
       res: 1.0*diff(w(x, y), x, y)
     Coeff: -2*x/y
         d: diff(z(x, y), x, x)
       out: -2*x/y
       res: -2*x*diff(z(x, y), x, x)/y
     Coeff: -x/y^2
         d: diff(w(x, y), x)
       out: -x/y^2
       res: -x*diff(w(x, y), x)/y^2


-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

In [21]:
f4 = Differential_Polynomial(diff(w, x,y) + diff(z, x,y) + diff(w, y)/(2*y) - diff(w,x)/y + x* diff(z, y)/y - w(x,y)/(2*y**2), ctx)
f4._orig
f4._p

This is the divisor: 1
     Coeff: 1
         d: diff(w(x, y), x, y)
       out: 1.0
       res: 1.0*diff(w(x, y), x, y)
     Coeff: 1
         d: diff(z(x, y), x, y)
       out: 1.0
       res: 1.0*diff(z(x, y), x, y)
     Coeff: -1/y
         d: diff(w(x, y), x)
       out: -1/y
       res: -diff(w(x, y), x)/y
     Coeff: 1/2/y
         d: diff(w(x, y), y)
       out: 1/2/y
       res: 1/2*diff(w(x, y), y)/y
     Coeff: x/y
         d: diff(z(x, y), y)
       out: x/y
       res: x*diff(z(x, y), y)/y
     Coeff: -1/2*w(x, y)/y^2
         d: 1
       out: -1/2*w(x, y)/y^2
       res: -1/2*w(x, y)/y^2


[<pylie.JanetBasis.DTerm object at 0x7feea2f2ba30>,
 <pylie.JanetBasis.DTerm object at 0x7feea2f2b2b0>,
 <pylie.JanetBasis.DTerm object at 0x7feea2f2b310>,
 <pylie.JanetBasis.DTerm object at 0x7feea2f2b940>,
 <pylie.JanetBasis.DTerm object at 0x7feea2f2bac0>,
 <pylie.JanetBasis.DTerm object at 0x7feea2f2b0d0>]

In [22]:
f5 = Differential_Polynomial(diff(w,y,y) + diff(z,x,y) - diff(w, y)/y + w(x,y)/y**2, ctx)
f5._orig

This is the divisor: 1
     Coeff: 1
         d: diff(w(x, y), y, y)
       out: 1.0
       res: 1.0*diff(w(x, y), y, y)
     Coeff: 1
         d: diff(z(x, y), x, y)
       out: 1.0
       res: 1.0*diff(z(x, y), x, y)
     Coeff: -1/y
         d: diff(w(x, y), y)
       out: -1/y
       res: -diff(w(x, y), y)/y
     Coeff: w(x, y)/y^2
         d: 1
       out: w(x, y)/y^2
       res: w(x, y)/y^2


-diff(w(x, y), y)/y + w(x, y)/y^2 + diff(w(x, y), y, y) + diff(z(x, y), x, y)

In [23]:
def Autoreduce(S, context):  
    i = 0
    while True:
        p = S[i]
        print ("While")
        p._orig.show()
     
        r = S[i+1:]
        if not r:
            S = Reorder(S, context,True)
            return S[:]    
        r[0]._orig.show()
        have_reduced = False
        checkidx = i+1
        for _r in r:
            print ("Try to reduce r{} with r{}".format(checkidx, i))
            res = reduce (_r, p, context)
            if res._orig != _r._orig:
                print ("have reduced!")
                res._orig.show()
                have_reduced = True                
            S[i] = res
            checkidx += 1
        S = Reorder(S, context, ascending = True)
        print("This is the new System........")
        for _ in S:
            _._orig.show()
        if not have_reduced:
            # no reduction done
            i += 1        
        else:
            i = 0
            

In [24]:
def JanetBasis(S, context):
    """List of homogenous PDE's + order context"""
    if not isinstance(S, Iterable):
        # bad criterion
        S = [S]
    S = [Differential_Polynomial(s, context) for s in S]
    S = Reorder(S, context)
    while 1:
        S = Autoreduce (S, context)
        #S = CompleteSystem (S, context)
        #f = FindIntegrableConditions(S, context)
        #c = ReduceIntegrabilityConditions(S, context)
        #if all ([_ == 0 for _ in c]) or not c:
        #    return S
        #S += c
        #S = Reorder (S, context)
        return S[:]

In [25]:
checkS=JanetBasis([f3._orig,f1._orig,f5._orig, f2._orig,f4._orig], ctx)

This is the divisor: 1
     Coeff: 1
         d: diff(w(x, y), x, y)
       out: 1.0
       res: 1.0*diff(w(x, y), x, y)
     Coeff: -2*x/y
         d: diff(z(x, y), x, x)
       out: -2*x/y
       res: -2*x*diff(z(x, y), x, x)/y
     Coeff: -x/y^2
         d: diff(w(x, y), x)
       out: -x/y^2
       res: -x*diff(w(x, y), x)/y^2
This is the divisor: 1
     Coeff: 1
         d: diff(w(x, y), y)
       out: 1.0
       res: 1.0*diff(w(x, y), y)
     Coeff: 1/2*x/((x^2 + y)*y)
         d: diff(z(x, y), y)
       out: 1/2*x/((x^2 + y)*y)
       res: 1/2*x*diff(z(x, y), y)/((x^2 + y)*y)
     Coeff: -w(x, y)/y
         d: 1
       out: -w(x, y)/y
       res: -w(x, y)/y
This is the divisor: 1
     Coeff: 1
         d: diff(w(x, y), y, y)
       out: 1.0
       res: 1.0*diff(w(x, y), y, y)
     Coeff: 1
         d: diff(z(x, y), x, y)
       out: 1.0
       res: 1.0*diff(z(x, y), x, y)
     Coeff: -1/y
         d: diff(w(x, y), y)
       out: -1/y
       res: -diff(w(x, y), y)/y
     Coeff: w

-2*x*diff(z(x, y), x, x)/y - x*diff(w(x, y), x)/y^2 + diff(w(x, y), x, y)

x*diff(z(x, y), y)/y - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + diff(w(x, y), x, y) + diff(z(x, y), x, y)

Try to reduce r1 with r0
This is the divisor: 2.0*x/y
     Coeff: 2.0*x/y
         d: diff(z(x, y), x, x)
       out: 1.0
       res: 1.0*diff(z(x, y), x, x)
     Coeff: 1.0
         d: diff(z(x, y), x, y)
       out: 0.5*y/x
       res: 0.5*y*diff(z(x, y), x, y)/x
     Coeff: 1.0*x/y^2 - 1/y
         d: diff(w(x, y), x)
       out: 0.5*y*(1.0*x/y^2 - 1/y)/x
       res: 0.5*y*(1.0*x/y^2 - 1/y)*diff(w(x, y), x)/x
     Coeff: 1/2/y
         d: diff(w(x, y), y)
       out: 0.25/x
       res: 0.25*diff(w(x, y), y)/x
     Coeff: x/y
         d: diff(z(x, y), y)
       out: 0.5
       res: 0.5*diff(z(x, y), y)
     Coeff: -1/2*w(x, y)/y^2
         d: 1
       out: -0.25*w(x, y)/(x*y)
       res: -0.25*w(x, y)/(x*y)
have reduced!


2.0*x*diff(z(x, y), x, x)/y + x*diff(z(x, y), y)/y + 1.0*x*diff(w(x, y), x)/y^2 - diff(w(x, y), x)/y + 1/2*diff(w(x, y), y)/y - 1/2*w(x, y)/y^2 + 1.0*diff(z(x, y), x, y)

...

-diff(w(x, y), y)/y + 1/2*x*diff(z(x, y), y, y)/((x^2 + y)*y) + w(x, y)/y^2 - 1/2*x*diff(z(x, y), y)/((x^2 + y)*y^2) - 1/2*x*diff(z(x, y), y)/((x^2 + y)^2*y) + diff(w(x, y), y, y)

-0.5*x*diff(z(x, y), y, y)/((x^2 + y)*y) + 0.5*x*diff(z(x, y), y)/((x^2 + y)*y^2) + 0.5*x*diff(z(x, y), y)/((x^2 + y)^2*y) + diff(z(x, y), x, y)

(2*x^2*y^3*diff(z(x, y), x, y) + y^4*diff(z(x, y), x, y) + 0.5*x^3*diff(z(x, y), y) + (x^4*diff(z(x, y), x, y) - 0.5*x*diff(z(x, y), y, y))*y^2 + (-0.5*x^3*diff(z(x, y), y, y) + 1.0*x*diff(z(x, y), y))*y)/(x^4*y^2 + 2*x^2*y^3 + y^4)

This is the divisor: 1.0*(x^4*diff(z(x, y), x, y) - 0.5*x*diff(z(x, y), y, y))*(-0.5*x^3*diff(z(x, y), y, y) + 1.0*x*diff(z(x, y), y))*x^5*y^10*diff(z(x, y), x, y)^2*diff(z(x, y), y) - x^4*y^2 - 2*x^2*y^3 - y^4
     Coeff: 1.0*(x^4*diff(z(x, y), x, y) - 0.5*x*diff(z(x, y), y, y))*(-0.5*x^3*diff(z(x, y), y, y) + 1.0*x*diff(z(x, y), y))*x^5*y^10*diff(z(x, y), x, y)^2*diff(z(x, y), y) - x^4*y^2 - 2*x^2*y^3 - y^4
         d: 1
       out: 1
       res: 1


ValueError: invalid expression '1' for DTerm

In [ ]:
for i in (f1._orig, f2._orig,f3._orig, f4._orig,f5._orig):
    i.show()

In [ ]:
for _ in checkS:
    _._orig.show()

In [ ]:
g1 = Differential_Polynomial (diff(z,y,y) + diff(z,y)/(2*y), ctx)
g1._orig

In [ ]:
g2 = Differential_Polynomial(diff(w,x,x) + 4*y*y*diff(w,y) - 8*y*y*diff(z, x) - 8*y*w(x,y), ctx)
g2._orig

In [ ]:
g3 = Differential_Polynomial(diff(w,x,y) - diff(z,x,x)/2 - diff(w,x)/(2*y) - 6*y**2*diff(z,y), ctx)
g3._orig

In [ ]:
g4 = Differential_Polynomial(diff(w,y,y) - 2*diff(z,x,y)- diff(w,y)/(2*y) + w(x,y)/(2*y**2), ctx)
g4._orig

In [ ]:
f1._orig

In [ ]:
f4._orig

In [ ]:
print(f4.Lder(), f1.Lder())

In [ ]:
f4a = reduce(f4, f1, ctx)

In [ ]:
for _ in f4a._p:
   print (_)

In [ ]:
f4a._orig

In [ ]:
wuwu=f4a._p[-1]._coeff.operands()[0]*f4a._p[-1]._coeff.operands()[1].expand().simplify_full()

In [ ]:
f4b = reduce(f4a, f1, ctx)

In [ ]:
f4b._orig

In [ ]:
functools.reduce(operator.mul, f4b._p[1]._coeff.operands()).expand().simplify_full()

In [ ]:
f4c = reduce(f4b, f2, ctx)

In [ ]:
f4c._p[2]._coeff.expand().simplify_full()

In [ ]:
f4d = reduce (f4c, f1, ctx)

In [ ]:
f4d._p[1]._coeff.expand().simplify_full()

In [ ]:
f4d._p[0]._d

In [ ]:
wuwu.collect(y)
# correct according Example 2.35

In [ ]:
wuwu=f4a._p[-2]._coeff

In [ ]:
wuwu

In [ ]:
wuwu=f4a._p[-2]._coeff.operands()

In [ ]:
wuwu

In [ ]:
wawa=wuwu[0]*wuwu[1]

In [ ]:
wawa.expand().simplify_full()

In [ ]:
# wawa is a